In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm


In [2]:
# !pip install opencv-python
# !pip install tqdm


In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
# Path to dataset
data_dir = "/content/drive/MyDrive/chest_xray/chest_xray/train"


In [6]:
import cv2
from PIL import Image

from google.colab import drive
drive.mount('/content/drive')

data_dir = "/content/drive/MyDrive/chest_xray/train"
class_folder = os.path.join(data_dir, "NORMAL")

image_files = os.listdir(class_folder)
first_image_path = os.path.join(class_folder, image_files[0])

from PIL import Image
image = Image.open(first_image_path)
print(image.size)
image.show()


# # Check if image was loaded successfully
# if image is not None:
#     # Resize (optional, for display purposes)
#     image_resized = cv2.resize(image, (512, 512))

#     # Show the image
#     cv2.imshow("Chest X-ray - NORMAL", image_resized)
#     cv2.waitKey(5000)  # Wait for key press
#     cv2.destroyAllWindows()
# else:
#     print("Failed to load image.")


Mounted at /content/drive
(1964, 1611)


In [7]:
import os
from PIL import Image
import numpy as np

def get_mean_std(folders):
    means = []
    stds = []
    count = 0

    for folder_path in folders:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    img_path = os.path.join(root, file)

                    # Open image in grayscale
                    img = Image.open(img_path).convert('L')  # 'L' = grayscale

                    img_np = np.array(img) / 255.0  # Normalize pixel values to [0,1]

                    means.append(np.mean(img_np))
                    stds.append(np.std(img_np))

                    count += 1
                    if count % 100 == 0:
                        print(f"Processed {count} images...")

    # Calculate dataset-wide mean and std
    dataset_mean = np.mean(means)
    dataset_std = np.mean(stds)

    return dataset_mean, dataset_std

# Define paths
base_dir = "/content/drive/MyDrive/chest_xray/chest_xray"
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

all_dirs = [train_dir, val_dir, test_dir]

mean_all, std_all = get_mean_std(all_dirs)

print(f"Combined Mean: {mean_all:.4f}, Combined Std: {std_all:.4f}")


Processed 100 images...
Processed 200 images...
Processed 300 images...
Processed 400 images...
Processed 500 images...
Processed 600 images...
Processed 700 images...
Processed 800 images...
Processed 900 images...
Processed 1000 images...
Processed 1100 images...
Processed 1200 images...
Processed 1300 images...
Processed 1400 images...
Processed 1500 images...
Processed 1600 images...
Processed 1700 images...
Processed 1800 images...
Processed 1900 images...
Processed 2000 images...
Processed 2100 images...
Processed 2200 images...
Processed 2300 images...
Processed 2400 images...
Processed 2500 images...
Processed 2600 images...
Processed 2700 images...
Processed 2800 images...
Processed 2900 images...
Processed 3000 images...
Processed 3100 images...
Processed 3200 images...
Processed 3300 images...
Processed 3400 images...
Processed 3500 images...
Processed 3600 images...
Processed 3700 images...
Processed 3800 images...
Processed 3900 images...
Processed 4000 images...
Processed

In [8]:
# Unified transform for all datasets (with minimal augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.481, 0.481, 0.481], [0.223, 0.223, 0.223]),
])


In [9]:
# define paths
data_dir = "/content/drive/MyDrive/chest_xray/chest_xray"


In [10]:
# Load datasets
#torchvision.datasets.ImageFolder is a utility class in PyTorch's torchvision library designed for loading image datasets that are organized in a specific directory structure. It simplifies the process of creating a PyTorch Dataset object from a collection of image files.
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=transform)
    for x in ['train', 'val', 'test']
}

In [11]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 5216
     Root location: /content/drive/MyDrive/chest_xray/chest_xray/train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
                Grayscale(num_output_channels=3)
                ToTensor()
                Normalize(mean=[0.481, 0.481, 0.481], std=[0.223, 0.223, 0.223])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 16
     Root location: /content/drive/MyDrive/chest_xray/chest_xray/val
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
                Grayscale(num_output_channels=3)
                ToTensor()
                Normalize(mean=[0.481, 0.481, 0.481], std=[0.223, 0.223, 0.223])
            ),
 'test': Dataset ImageFolder
     Number of datapoints: 624
     Root location: /content/drive/MyDrive/chest_xray/ches

In [12]:
class_names = image_datasets['train'].classes
print(f"Classes: {class_names}")

Classes: ['NORMAL', 'PNEUMONIA']


In [13]:
# Load dataloaders
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x == 'train'), num_workers=2)
    for x in ['train', 'val', 'test']
}

In [14]:
import torch
import torchvision.models as models

# === Define your model architecture ===
model = models.densenet169(pretrained=False)

# Modify first conv layer to accept 1-channel input (grayscale)
# because your transform uses Grayscale(num_output_channels=1)
model.features.conv0 = torch.nn.Conv2d(
    1, 64, kernel_size=7, stride=2, padding=3, bias=False
)

# (Optional) Modify classifier to match your number of classes, e.g. 2
num_features = model.classifier.in_features
model.classifier = torch.nn.Linear(num_features, 2)

print("✅ Model initialized successfully")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Model initialized successfully


In [15]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# === 1. Set your dataset base path ===
# Use the same path you used during training.
# For example, if your train/val/test folders were under this path:
BASE_DIR = "/content/drive/MyDrive/chest_xray/chest_xray"  # change if different

# === 2. Build test transform to match model (1-channel grayscale) ===
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),   # 1-channel input for DenseNet169
    transforms.ToTensor(),
    transforms.Normalize([0.481], [0.223]),
])

# === 3. Rebuild test dataset and loader ===
TEST_DIR = f"{BASE_DIR}/test"
test_ds = datasets.ImageFolder(TEST_DIR, transform=test_transform)
dataloaders['test'] = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# === 4. Sanity checks ===
xb, yb = next(iter(dataloaders['test']))
print("✅ BASE_DIR:", BASE_DIR)
print("✅ Test batch shape:", xb.shape)   # should be [batch, 1, 224, 224]
print("✅ Model input channels:", model.features.conv0.in_channels if hasattr(model, "features") else model.conv1.in_channels)


✅ BASE_DIR: /content/drive/MyDrive/chest_xray/chest_xray
✅ Test batch shape: torch.Size([32, 1, 224, 224])
✅ Model input channels: 1


In [16]:
model = models.densenet169(weights=models.DenseNet169_Weights.DEFAULT)

# Replace first conv to accept 1 channel
old = model.features.conv0
model.features.conv0 = nn.Conv2d(
    1, old.out_channels,
    kernel_size=old.kernel_size,
    stride=old.stride,
    padding=old.padding,
    bias=False
)

# Replace classifier for 2 classes
model.classifier = nn.Linear(model.classifier.in_features, 2)

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


100%|██████████| 54.7M/54.7M [00:00<00:00, 125MB/s]


In [17]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [18]:
# Modify first conv layer for 1-channel (grayscale)
model = models.densenet169(weights=models.DenseNet169_Weights.DEFAULT)
model.features.conv0 = nn.Conv2d(
    1, model.features.conv0.out_channels,
    kernel_size=model.features.conv0.kernel_size,
    stride=model.features.conv0.stride,
    padding=model.features.conv0.padding,
    bias=False
)


In [19]:
# Modify final FC layer for 2 classes
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)


In [20]:
num_ftrs

1664

In [21]:
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

In [22]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [23]:
# Continue with training loop
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50


In [24]:
# Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=50):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-" * 30)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase], desc=phase):
                inputs, labels = inputs.to(device), labels.to(device)

                # Convert to grayscale (1 channel) if needed
                if inputs.shape[1] != 1:
                    inputs = inputs.mean(dim=1, keepdim=True)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    print("\nTraining Complete!")

In [25]:
train_model(model=model, dataloaders=dataloaders, criterion=criterion, optimizer=optimizer, num_epochs=50)


Epoch 1/50
------------------------------


train: 100%|██████████| 163/163 [01:52<00:00,  1.46it/s]


train Loss: 0.1452 Acc: 0.9459


val: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s]


val Loss: 1.8367 Acc: 0.5000

Epoch 2/50
------------------------------


...

val Loss: 0.0882 Acc: 0.9375

Epoch 50/50
------------------------------


train: 100%|██████████| 163/163 [01:48<00:00,  1.50it/s]


train Loss: 0.0000 Acc: 1.0000


val: 100%|██████████| 1/1 [00:00<00:00,  2.22it/s]

val Loss: 0.0627 Acc: 0.9375

Training Complete!


In [26]:
# Evaluation on test set
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Testing'):
            inputs, labels = inputs.to(device), labels.to(device)

            if inputs.shape[1] != 1:
                inputs = inputs.mean(dim=1, keepdim=True)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    print(f"\nTest Accuracy: {100 * correct / total:.2f}%")


In [27]:
save_path = '/content/drive/MyDrive/chest_xray/chest_xray/weights/densenet169_pretrained.pth'


In [28]:
torch.save({
    "state_dict": model.state_dict(),
    "arch": "densenet169",
    "input_channels": 1,   # or 3
    "num_classes": 2
}, "/content/drive/MyDrive/chest_xray/chest_xray/weights/densenet169_best.pth")

In [31]:
import os, torch, torch.nn as nn
from torchvision import models

# --- 1) Where your raw checkpoint lives (update if needed)
ckpt_path = "/content/drive/MyDrive/chest_xray/chest_xray/weights/densenet169_best.pth"

# --- 2) Load raw checkpoint (map to CPU for portability)
ckpt = torch.load(ckpt_path, map_location="cpu")

# Handle different checkpoint formats
sd = ckpt.get("state_dict") or ckpt.get("model_state_dict") or ckpt
assert isinstance(sd, dict), "No state_dict found in checkpoint."

# --- 3) Normalize key names
new_sd = {}
for k, v in sd.items():
    k2 = k
    if k2.startswith("module."):           # from DataParallel
        k2 = k2[len("module."):]
    if k2.startswith("fc."):               # some code used resnet-style naming
        k2 = k2.replace("fc.", "classifier.", 1)
    new_sd[k2] = v

# --- 4) Recreate architecture exactly
m = models.densenet169(weights=None)
# if training was grayscale (1 channel), swap first conv
input_ch = ckpt.get("input_channels", 3)
if input_ch == 1:
    old = m.features.conv0
    m.features.conv0 = nn.Conv2d(1, old.out_channels,
                                 kernel_size=old.kernel_size,
                                 stride=old.stride, padding=old.padding,
                                 bias=False)

# match classifier num_classes if stored
num_classes = ckpt.get("num_classes", m.classifier.out_features)
if m.classifier.out_features != num_classes:
    m.classifier = nn.Linear(m.classifier.in_features, num_classes)

# --- 5) Load weights
missing, unexpected = m.load_state_dict(new_sd, strict=False)
print("Loaded with:\n  missing:", missing, "\n  unexpected:", unexpected)

# sanity: ensure at least the main blocks loaded
assert any(k.startswith("features.denseblock1") for k in new_sd.keys()), "Weights don't look like DenseNet."

# --- 6) (Optional) move to CUDA for inference now
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
m.to(device).eval()

# --- 7) Save a clean, portable copy for VS Code
os.makedirs("saved_models", exist_ok=True)
clean_path = "saved_models/densenet169_clean.pth"
torch.save(m.state_dict(), clean_path)
print(f"✅ Saved cleaned weights to: {clean_path}")

# (Optional) also save full model (architecture+weights) – less flexible, but one-liner load
full_path = "saved_models/densenet169_fullmodel.pth"
torch.save(m, full_path)
print(f"✅ Saved full model to: {full_path}")



Loaded with:
  missing: [] 
  unexpected: []
✅ Saved cleaned weights to: saved_models/densenet169_clean.pth
✅ Saved full model to: saved_models/densenet169_fullmodel.pth


In [32]:
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [33]:
# correct = 0
# total = 0

# with torch.no_grad():  # no need to track gradients
#     for images, labels in val_loader:
#         images, labels = images.to(device), labels.to(device)

#         outputs = model(images)  # shape: [batch_size, num_classes]
#         _, predicted = torch.max(outputs, 1)  # get predicted class indices

#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = 100 * correct / total
# print(f'Validation Accuracy: {accuracy:.2f}%')

In [34]:
# build model ...
# (if you replace conv0 / classifier, do it here)

# load checkpoint if any ...
# model.load_state_dict(...)

model = model.to(device)      # <<< IMPORTANT
model.eval()


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [35]:
# Evaluation on test set
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Testing'):
            inputs, labels = inputs.to(device), labels.to(device)

            if inputs.shape[1] != 1:
                inputs = inputs.mean(dim=1, keepdim=True)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    print(f"\nVal Accuracy: {100 * correct / total:.2f}%")

In [36]:
@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    total, correct = 0, 0
    for x, y in dataloader:
        x = x.to(device)      # <<< move batch
        y = y.to(device)
        out = model(x)
        pred = out.argmax(1)
        total += y.size(0)
        correct += (pred == y).sum().item()
    print(f"Accuracy: {100*correct/total:.2f}%")


In [37]:
evaluate(model, dataloaders['test'])

Accuracy: 79.33%


In [38]:
import os

path = "/content/drive/MyDrive/chest_xray"
print(os.listdir(path))


['__MACOSX', 'chest_xray', 'test', 'train', 'val']
